## 4.2. Przetwarzanie wstępne obrazów

#### Importowanie bibliotek

In [1]:
import cv2
from pathlib import Path

#### Funkcja `enhance_contrast`
Wzmocnienie kontrastu obrazu za pomocą CLAHE w przestrzeni LAB.

In [2]:
def enhance_contrast(image, clipLimit=0.85, tileGridSize=(4,4)):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

#### Funkcja `remove_hair`
Usuwanie włosów za pomocą operacji blackhat i inpaintingu.

In [3]:
def remove_hair(image, kernel_size=(7,7), inpaint_radius=5):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    blackhat = cv2.morphologyEx(blur, cv2.MORPH_BLACKHAT, kernel)
    _, mask = cv2.threshold(blackhat, 0, 255, cv2.THRESH_OTSU)
    return cv2.inpaint(image, mask, inpaint_radius, cv2.INPAINT_TELEA)

#### Funkcja `preprocess_image`
Pełny preprocessing obrazu: CLAHE + usuwanie włosów.

In [4]:
def preprocess_image(image):
    img1 = enhance_contrast(image)
    img2 = remove_hair(img1)
    return img2

#### Funkcja `preprocess_folder`
Przetwarzanie wszystkich obrazów z wskazanych katalogów `input_dirs` i zapisywanie do `output_dir`.

In [5]:
def preprocess_folders(input_dirs, output_dir):
    output_path = Path(output_dir)
    
    for root in input_dirs:
        input_path = Path(root)
        for img_file in input_path.rglob('*.*'):
            
            if not img_file.is_file():
                continue
            img = cv2.imread(str(img_file))
            
            if img is None:
                print(f'Nie wczytano {img_file}')
                continue
            
            proc = preprocess_image(img)

            rel = img_file.relative_to(input_path)
            target = output_path / rel
            target.parent.mkdir(parents=True, exist_ok=True)
            cv2.imwrite(str(target), proc)